In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Show all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr" # change to all, to show all

# Trade Data

In [2]:
data = pd.read_pickle("full_trade.pickle")
print(data.shape)
data.head()

(240192, 6)


,reporter,product,indicators,TIME_PERIOD,OBS_VALUE,flags
0,AT,2207,QUANTITY_IN_100KG,1988-01-01,NaN,:
1,AT,2207,VALUE_IN_EUROS,1988-01-01,NaN,:
2,AT,291811,QUANTITY_IN_100KG,1988-01-01,NaN,:
3,AT,291811,VALUE_IN_EUROS,1988-01-01,NaN,:
4,AT,291719,QUANTITY_IN_100KG,1988-01-01,NaN,:


In [3]:
data = data.rename(columns={'product':'products'})


'440130':"Sawdust" only appears until the end of 2011, '440139':"Sawdust" appears after

In [4]:
data = data[(data.products != '440130') | (data.TIME_PERIOD < "2012")]
data = data[(data.products != '440139') | (data.TIME_PERIOD >= "2012")]

In [5]:
data[(data.products == '440139')]

,reporter,products,indicators,TIME_PERIOD,OBS_VALUE,flags
145162,AT,440139,QUANTITY_IN_100KG,2012-01-01,379460.39,
145163,AT,440139,VALUE_IN_EUROS,2012-01-01,2450542.00,
145180,BE,440139,QUANTITY_IN_100KG,2012-01-01,1429626.00,
145181,BE,440139,VALUE_IN_EUROS,2012-01-01,11656331.00,
145198,BG,440139,QUANTITY_IN_100KG,2012-01-01,4.55,
...,...,...,...,...,...,...
217685,SE,440139,VALUE_IN_EUROS,2023-12-01,NaN,:
217702,SI,440139,QUANTITY_IN_100KG,2023-12-01,NaN,:
217703,SI,440139,VALUE_IN_EUROS,2023-12-01,NaN,:
217720,SK,440139,QUANTITY_IN_100KG,2023-12-01,NaN,:


In [6]:
prods = {
    "2207":"Ethanol", "291811":"LA", "291719":"SA",
    '440130':"Sawdust", '440139':"Sawdust",
    '1213':"Straw", '230210':"Maize", '230230':"Wheat",
    '230320': "Beet",
    }
data = data[data.products.isin(prods.keys())]
data = data.replace({"products": prods})
print(data.shape)
data.head()

(191808, 6)


,reporter,products,indicators,TIME_PERIOD,OBS_VALUE,flags
0,AT,Ethanol,QUANTITY_IN_100KG,1988-01-01,NaN,:
1,AT,Ethanol,VALUE_IN_EUROS,1988-01-01,NaN,:
2,AT,LA,QUANTITY_IN_100KG,1988-01-01,NaN,:
3,AT,LA,VALUE_IN_EUROS,1988-01-01,NaN,:
4,AT,SA,QUANTITY_IN_100KG,1988-01-01,NaN,:


In [7]:
data["flags"].value_counts()

flags
     139277
:     52531
Name: count, dtype: int64

In [8]:
data.isna().sum()

reporter           0
products           0
indicators         0
TIME_PERIOD        0
OBS_VALUE      52531
flags              0
dtype: int64

NaN are not marked as not available in eurostat


In [9]:
data = data[
    ['reporter', 'products', 'indicators', 'TIME_PERIOD', 'OBS_VALUE',
    #'flags'
    ]
       ]
data.head()

,reporter,products,indicators,TIME_PERIOD,OBS_VALUE
0,AT,Ethanol,QUANTITY_IN_100KG,1988-01-01,NaN
1,AT,Ethanol,VALUE_IN_EUROS,1988-01-01,NaN
2,AT,LA,QUANTITY_IN_100KG,1988-01-01,NaN
3,AT,LA,VALUE_IN_EUROS,1988-01-01,NaN
4,AT,SA,QUANTITY_IN_100KG,1988-01-01,NaN


In [10]:
countries = [
    'BE', 'BG', 'DE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HU', 'IT', 'NL', 'RO', 'SE', 'PL'
]
data = data[data.reporter.isin(countries)]
data = data[data.TIME_PERIOD <= "2023-05"]
data.head()

,reporter,products,indicators,TIME_PERIOD,OBS_VALUE
18,BE,Ethanol,QUANTITY_IN_100KG,1988-01-01,57974.0
19,BE,Ethanol,VALUE_IN_EUROS,1988-01-01,3040780.0
20,BE,LA,QUANTITY_IN_100KG,1988-01-01,659.0
21,BE,LA,VALUE_IN_EUROS,1988-01-01,145528.0
22,BE,SA,QUANTITY_IN_100KG,1988-01-01,3532.0


In [11]:
print(data.shape)
data.isna().sum()

(95200, 5)


reporter           0
products           0
indicators         0
TIME_PERIOD        0
OBS_VALUE      18156
dtype: int64

In [12]:
data[data.products == "Sawdust"].sort_values('TIME_PERIOD')

,reporter,products,indicators,TIME_PERIOD,OBS_VALUE
26,BE,Sawdust,QUANTITY_IN_100KG,1988-01-01,222047.00
459,SE,Sawdust,VALUE_IN_EUROS,1988-01-01,NaN
458,SE,Sawdust,QUANTITY_IN_100KG,1988-01-01,NaN
441,RO,Sawdust,VALUE_IN_EUROS,1988-01-01,NaN
440,RO,Sawdust,QUANTITY_IN_100KG,1988-01-01,NaN
...,...,...,...,...,...
213725,BE,Sawdust,VALUE_IN_EUROS,2023-05-01,1682268.00
213724,BE,Sawdust,QUANTITY_IN_100KG,2023-05-01,125638.85
214156,SE,Sawdust,QUANTITY_IN_100KG,2023-05-01,254129.09
213904,GB,Sawdust,QUANTITY_IN_100KG,2023-05-01,NaN


In [13]:
df1 = data[data.indicators == "QUANTITY_IN_100KG"].drop("indicators", axis = 1).rename(columns={"OBS_VALUE" : "QUANTITY_IN_100KG"})
df2 = data[data.indicators == "VALUE_IN_EUROS"].drop("indicators", axis = 1).rename(columns={"OBS_VALUE" : "VALUE_IN_EUROS"})
df = df1.merge(df2, on =["reporter", "products", "TIME_PERIOD"], how="outer")
print(df.shape)
df.head()

(47600, 5)


,reporter,products,TIME_PERIOD,QUANTITY_IN_100KG,VALUE_IN_EUROS
0,BE,Ethanol,1988-01-01,57974.0,3040780.0
1,BE,LA,1988-01-01,659.0,145528.0
2,BE,SA,1988-01-01,3532.0,580792.0
3,BE,Sawdust,1988-01-01,222047.0,867077.0
4,BE,Straw,1988-01-01,120146.0,684618.0


In [14]:
df.isna().sum()

reporter                0
products                0
TIME_PERIOD             0
QUANTITY_IN_100KG    9080
VALUE_IN_EUROS       9076
dtype: int64

In [15]:
feedstocks = ["Sawdust","Straw","Maize","Wheat", "Beet"]
products = ["Ethanol", "LA", "SA"]
fdf = df[df.products.isin(feedstocks)]
pdf = df[df.products.isin(products)]
fdf.head()

,reporter,products,TIME_PERIOD,QUANTITY_IN_100KG,VALUE_IN_EUROS
3,BE,Sawdust,1988-01-01,222047.0,867077.0
4,BE,Straw,1988-01-01,120146.0,684618.0
5,BE,Maize,1988-01-01,23476.0,370315.0
6,BE,Wheat,1988-01-01,64086.0,902319.0
10,BG,Sawdust,1988-01-01,NaN,NaN


In [16]:
@interact
def mna(Product=list(fdf.products.unique()), Country = ["All"] + list(fdf.reporter.unique())): 
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    aux = fdf[fdf.reporter == Country].copy() if Country != "All" else fdf.copy()
    aux = aux[(aux.products == Product)]
    print(" "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ","NaN Values:", aux.QUANTITY_IN_100KG.isna().sum(), "of", len(aux.QUANTITY_IN_100KG), f"({round(aux.QUANTITY_IN_100KG.isna().sum()/len(aux.QUANTITY_IN_100KG)*100)}%)")
    auxna = aux[aux.QUANTITY_IN_100KG.isna()]
    auxna = auxna.fillna(0)
    aux = aux[~aux.QUANTITY_IN_100KG.isna()]
    if Country == "All":
            sns.lineplot(aux, x= "TIME_PERIOD", y ="QUANTITY_IN_100KG", hue = "reporter", ax = ax)
    else:
        sns.lineplot(aux, x= "TIME_PERIOD", y ="QUANTITY_IN_100KG", ax = ax, label = "Values", color = "#23708a")
        sns.scatterplot(auxna, x= "TIME_PERIOD", y ="QUANTITY_IN_100KG", ax = ax, color = "#4f9f8d", markers = "True" , label= "Missing Data")
        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)




interactive(children=(Dropdown(description='Product', options=('Sawdust', 'Straw', 'Maize', 'Wheat', 'Beet'), …

In [17]:
final_f = ["Beet", "Wheat", "Maize"]
final_fc = ["FR", "BE", "ES"]
final_fdf = fdf.loc[fdf.products.isin(final_f) & fdf.reporter.isin(final_fc), ["reporter", "products", "TIME_PERIOD", "QUANTITY_IN_100KG", "VALUE_IN_EUROS"]]
print(final_fdf.shape)
final_fdf.isna().sum()


(3825, 5)


reporter              0
products              0
TIME_PERIOD           0
QUANTITY_IN_100KG    72
VALUE_IN_EUROS       71
dtype: int64

In [18]:
final_fdf.loc[:, "Tons"] = final_fdf.QUANTITY_IN_100KG / 10
final_fdf.loc[:, "Value"] = final_fdf.VALUE_IN_EUROS
final_fdf.loc[:, "Price"] = final_fdf.VALUE_IN_EUROS / final_fdf.Tons
final_fdf.isna().sum()

reporter              0
products              0
TIME_PERIOD           0
QUANTITY_IN_100KG    72
VALUE_IN_EUROS       71
Tons                 72
Value                71
Price                72
dtype: int64

In [19]:
ffdf = final_fdf.drop(["QUANTITY_IN_100KG", "VALUE_IN_EUROS"], axis = 1)
ffdf.to_pickle("feedstocks.pickle")
ffdf.head()

,reporter,products,TIME_PERIOD,Tons,Value,Price
5,BE,Maize,1988-01-01,2347.6,370315.0,157.741949
6,BE,Wheat,1988-01-01,6408.6,902319.0,140.798146
26,ES,Maize,1988-01-01,NaN,NaN,NaN
27,ES,Wheat,1988-01-01,65.8,10503.0,159.620061
40,FR,Maize,1988-01-01,252.9,56086.0,221.771451


In [20]:
ffdf[ffdf.Tons.isna()]

,reporter,products,TIME_PERIOD,Tons,Value,Price
26,ES,Maize,1988-01-01,NaN,NaN,NaN
124,ES,Maize,1988-02-01,NaN,NaN,NaN
320,ES,Maize,1988-04-01,NaN,NaN,NaN
418,ES,Maize,1988-05-01,NaN,NaN,NaN
516,ES,Maize,1988-06-01,NaN,NaN,NaN
...,...,...,...,...,...,...
18156,ES,Maize,2003-06-01,NaN,NaN,NaN
41667,ES,Beet,1988-02-01,NaN,NaN,NaN
41779,ES,Beet,1988-10-01,NaN,204.0,NaN
41807,ES,Beet,1988-12-01,NaN,NaN,NaN


In [21]:
pdf = pdf[(pdf.products != "Ethanol") | pdf.reporter.isin(["DE", "NL", "FR"])]
pdf = pdf[(pdf.products != "LA") | pdf.reporter.isin(["DE", "NL", "BE"])]
pdf = pdf[(pdf.products != "SA") | pdf.reporter.isin(["DE", "NL", "IT"])]
pdf.head()


,reporter,products,TIME_PERIOD,QUANTITY_IN_100KG,VALUE_IN_EUROS
1,BE,LA,1988-01-01,659.0,145528.0
14,DE,Ethanol,1988-01-01,26854.0,1442300.0
15,DE,LA,1988-01-01,3422.0,504490.0
16,DE,SA,1988-01-01,7766.0,1431164.0
35,FR,Ethanol,1988-01-01,43174.0,2183328.0


In [22]:
pdf.isna().sum()

reporter             0
products             0
TIME_PERIOD          0
QUANTITY_IN_100KG    0
VALUE_IN_EUROS       0
dtype: int64

In [23]:
pdf.loc[:, "Tons"] = pdf.QUANTITY_IN_100KG / 10
pdf.loc[:, "Value"] = pdf.VALUE_IN_EUROS
pdf.loc[:, "Price"] = pdf.VALUE_IN_EUROS / pdf.Tons
pdf.isna().sum()

reporter             0
products             0
TIME_PERIOD          0
QUANTITY_IN_100KG    0
VALUE_IN_EUROS       0
Tons                 0
Value                0
Price                0
dtype: int64

In [24]:
pdf = pdf.drop(["QUANTITY_IN_100KG", "VALUE_IN_EUROS"], axis = 1)
pdf.to_pickle("products.pickle")
pdf.tail()

,reporter,products,TIME_PERIOD,Tons,Value,Price
41587,FR,Ethanol,2023-05-01,54717.730,74291700.0,1357.726280
41617,IT,SA,2023-05-01,1753.718,4920345.0,2805.664879
41622,NL,Ethanol,2023-05-01,110159.300,134098899.0,1217.318002
41623,NL,LA,2023-05-01,1290.495,2530419.0,1960.812711
41624,NL,SA,2023-05-01,771.981,2556345.0,3311.409219


# Feedstock Production

In [25]:
crops = pd.read_csv("crops.csv")
crops = crops[["crops", "geo", "TIME_PERIOD", "OBS_VALUE"]]
crops = crops.rename(columns={"OBS_VALUE" : "Mtons"})
crops.Mtons = crops.Mtons / 1000
crops = crops.replace({"crops": {
    'C1100' : "Wheat", 
    'G3000' : "Maize", 
    'R2000' : "Sugar Beet"
}})
crops.head()

,crops,geo,TIME_PERIOD,Mtons
0,Wheat,BE,2000,1.67583
1,Wheat,BE,2001,1.44723
2,Wheat,BE,2002,1.66926
3,Wheat,BE,2003,1.71652
4,Wheat,BE,2004,1.88641


In [26]:
waste_factors = {
    'Wheat' : (0.5 + 2.37) / 2,
    'Maize' : (0.8 + 3.77) / 2, 
    'Sugar Beet' : (0.14 + 0.91) / 2 + (2.68 + 5.15)/2
}
waste_factors


{'Wheat': 1.435, 'Maize': 2.285, 'Sugar Beet': 4.44}

In [27]:
crops["Mtons_Waste"] = crops.apply(lambda x: x.Mtons * waste_factors[x.crops], axis = 1)
crops.head()

,crops,geo,TIME_PERIOD,Mtons,Mtons_Waste
0,Wheat,BE,2000,1.67583,2.404816
1,Wheat,BE,2001,1.44723,2.076775
2,Wheat,BE,2002,1.66926,2.395388
3,Wheat,BE,2003,1.71652,2.463206
4,Wheat,BE,2004,1.88641,2.706998


In [28]:
@interact
def croptrends(Waste=["All"] + list(crops.crops.unique()), Country = ["All"] + list(crops.geo.unique())): 
    sns.set_palette("pastel")
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    aux = crops[crops.geo == Country].copy() if Country != "All" else fdf.copy()
    aux = crops[(crops.crops == Waste)].copy() if Waste != "All" else aux.copy()
    if Country == "All":
        if Waste != "All":
            print(" "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ","NaN Values:", aux.Mtons_Waste.isna().sum(), "of", len(aux.Mtons_Waste), f"({round(aux.Mtons_Waste.isna().sum()/len(aux.Mtons_Waste)*100)}%)")
            sns.lineplot(aux[aux.crops == Waste], x= "TIME_PERIOD", y ="Mtons_Waste", hue = "geo", ax = ax)
    else:
        if Waste == "All":
            print(" "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ","NaN Values:", aux.Mtons_Waste.isna().sum(), "of", len(aux.Mtons_Waste), f"({round(aux.Mtons_Waste.isna().sum()/len(aux.Mtons_Waste)*100)}%)")
            sns.lineplot(aux[aux.geo == Country], x= "TIME_PERIOD", y ="Mtons_Waste", hue = "crops", ax = ax)




interactive(children=(Dropdown(description='Waste', options=('All', 'Wheat', 'Maize', 'Sugar Beet'), value='Al…

In [70]:
fpdf = pdf.rename(columns={"reporter":"geo"})
fpdf.head()

,geo,products,TIME_PERIOD,Tons,Value,Price
1,BE,LA,1988-01-01,65.9,145528.0,2208.315630
14,DE,Ethanol,1988-01-01,2685.4,1442300.0,537.089447
15,DE,LA,1988-01-01,342.2,504490.0,1474.254822
16,DE,SA,1988-01-01,776.6,1431164.0,1842.858614
35,FR,Ethanol,1988-01-01,4317.4,2183328.0,505.704359


In [71]:
ffdf = ffdf.rename(columns= {"reporter":"geo", "products":"wastes"})
ffdf.head()

,geo,wastes,TIME_PERIOD,Tons,Value,Price
5,BE,Maize,1988-01-01,2347.6,370315.0,157.741949
6,BE,Wheat,1988-01-01,6408.6,902319.0,140.798146
26,ES,Maize,1988-01-01,NaN,NaN,NaN
27,ES,Wheat,1988-01-01,65.8,10503.0,159.620061
40,FR,Maize,1988-01-01,252.9,56086.0,221.771451


In [97]:
fcrops = crops[["crops", "geo", "TIME_PERIOD", "Mtons_Waste"]].rename(columns={"crops":"wastes", "Mtons_waste":"MTons"})
fcrops.head()

,wastes,geo,TIME_PERIOD,Mtons_Waste
0,Wheat,BE,2000,2.404816
1,Wheat,BE,2001,2.076775
2,Wheat,BE,2002,2.395388
3,Wheat,BE,2003,2.463206
4,Wheat,BE,2004,2.706998


In [98]:
ffdf.wastes.unique()

array(['Maize', 'Wheat', 'Beet'], dtype=object)

In [99]:
fcrops.loc[fcrops.wastes == 'Sugar Beet', "wastes" ] = "Beet"
fcrops.wastes.unique()

array(['Wheat', 'Maize', 'Beet'], dtype=object)

In [100]:
fcrops.head()

,wastes,geo,TIME_PERIOD,Mtons_Waste
0,Wheat,BE,2000,2.404816
1,Wheat,BE,2001,2.076775
2,Wheat,BE,2002,2.395388
3,Wheat,BE,2003,2.463206
4,Wheat,BE,2004,2.706998


In [85]:
fpdf.products.unique()

array(['LA', 'Ethanol', 'SA'], dtype=object)

# Dataset Creation

In [76]:
def create_dic_ts(data, field = "products", name = "tsdict"):
    dic = {}
    for i in data[field].unique():
        df = data[data[field] == i].drop(field, axis = 1)
        dic[i] = {}
        for country in df.geo.unique():
            dic[i][country] = df[df.geo == country].drop("geo", axis = 1).copy().set_index("TIME_PERIOD")

    with open(f'{name}.pickle', 'wb') as f:
        pickle.dump(dic, f)
    return dic



In [77]:
tsprod = create_dic_ts(fpdf, "products", "tsprod")
for k in tsprod.keys():
    print(f"{k}:", list(tsprod[k].keys()))
tsprod["Ethanol"]["DE"]

LA: ['BE', 'DE', 'NL']
Ethanol: ['DE', 'FR', 'NL']
SA: ['DE', 'IT', 'NL']


,Tons,Value,Price
TIME_PERIOD,,,
1988-01-01,2685.400,1442300.0,537.089447
1988-02-01,4195.200,2012402.0,479.691552
1988-03-01,4326.900,2296841.0,530.828307
1988-04-01,3629.100,1858971.0,512.240225
1988-05-01,5547.200,2682168.0,483.517450
...,...,...,...
2023-01-01,92008.369,102825277.0,1117.564392
2023-02-01,88966.693,103824197.0,1167.000745
2023-03-01,96104.346,124771296.0,1298.289840


In [78]:
tsfeed = create_dic_ts(ffdf, "wastes", "tsfeed")
for k in tsfeed.keys():
    print(f"{k}:", list(tsfeed[k].keys()))
tsfeed["Wheat"]["FR"]

Maize: ['BE', 'ES', 'FR']
Wheat: ['BE', 'ES', 'FR']
Beet: ['BE', 'ES', 'FR']


,Tons,Value,Price
TIME_PERIOD,,,
1988-01-01,94.500,13053.0,138.126984
1988-02-01,175.300,26795.0,152.852253
1988-03-01,152.600,31447.0,206.074705
1988-04-01,153.000,27390.0,179.019608
1988-05-01,31.200,9231.0,295.865385
...,...,...,...
2023-01-01,1687.679,630877.0,373.813385
2023-02-01,1431.005,424529.0,296.664931
2023-03-01,2325.583,640468.0,275.401050


In [101]:
tscrops = create_dic_ts(fcrops, "wastes", "tscrops")
for k in tscrops.keys():
    print(f"{k}:", list(tscrops[k].keys()))
tscrops["Wheat"]["FR"].head()

Wheat: ['BE', 'ES', 'FR']
Maize: ['BE', 'ES', 'FR']
Beet: ['BE', 'ES', 'FR']


,Mtons_Waste
TIME_PERIOD,
2000,52.978851
2001,44.734044
2002,55.219790
2003,43.231628
2004,56.296987


In [102]:
with open('tscrops.pickle', 'rb') as f:
    tscrops = pickle.load(f)
tscrops["Beet"]["FR"].head()

,Mtons_Waste
TIME_PERIOD,
2000,138.177240
2001,119.201124
2002,148.517556
2003,130.222980
2004,136.700052
